<a href="https://colab.research.google.com/github/NicoAleC/AutoRepoMKT/blob/main/ReporteMKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(len("Nicolas Alejandro Carrasco Arandia"))
!python --version
#https://medium.com/@lukas.forst/supercharge-your-word-reports-using-python-docx-template-4e9ebfc66b9e

34
Python 3.10.12


In [ ]:
#!sudo apt-get install python3.12
#!pip install -U pip
!pip install docxtpl
!pip install googletrans==3.1.0a0
#!pip install python-docx==1.1.0
#!pip install -U secure-smtplib
#costo de leads promedios, costos de publicidad y leads generados

In [ ]:
import gspread
import locale

import pandas as pd
import calendar
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate, InlineImage
from docx.shared import Mm

from datetime import datetime

from google.colab import auth, files
from google.auth import default
from googletrans import Translator

import re

import smtplib
import ssl
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

auth.authenticate_user()

creds, _ = default()
gc = gspread.authorize(creds)

translator = Translator()

In [ ]:
def getEMails(sede):
  gs = gc.open(sede).worksheet('eMails')
  data = gs.get_all_values()
  headers = data.pop(0)
  df = pd.DataFrame(data, columns=headers)
  return df

def getDataFrame(sede, registro):
  gs = gc.open(sede).worksheet(registro)
  data = gs.get_all_values()
  headers = data.pop(0)
  df = pd.DataFrame(data, columns=headers)
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.iloc[:,1:-7]
  df.drop(df[df['NOMBRE ASESOR(A)'] == ''].index, inplace = True)
  df = df.loc[:, ['PERIODO', # No entra a la tabla
              'NOMBRE ASESOR(A)', # No entra a la tabla
              'UNIDAD DE NEGOCIO CORRESPONDIENTE', # Entra por separado
              'NOMBRE PROGRAMA',
              'APELLIDO Y NOMBRE',
              'FECHA DE INSCRIPCION PORTAL',
              'PROMOCIONES',
              'MODALIDAD DE PAGOS',
              'CANTIDAD DE CUOTAS',
              'COLEGIATURA A RECOLECTAR (SOLO MONTO COLEGIATURA)',
              'M FECHA RECIBO',
              'M N° RECIBO',
              'M MONTO RECIBO',
              'C FECHA RECIBO',
              'C N° RECIBO',
              'C MONTO RECIBO',
              'CORRESP A META DE SUELDO?',
              'COMISION SEGUN TABLA',
              'COMISION PLAN CONTADO',
              'TOTAL COMISION',
              # datos para la tabla de documentos entregados
              'CI',
              'TITULO EN PROV NAL',
              'DIPLOMA ACADEMICO',
              'DECLARATORIA',
              'CARTA DE INSCRIPCIÓN',
              'FORMULARIO DE INSCRIPCIÓN',
              'CURRICULUM DOCUMENTADO',
              'ESTADO DE INSCRIPCIÓN',
            ]
      ]
  df['NOMBRE PROGRAMA'] = df['NOMBRE PROGRAMA'].apply(deleteSpaces)
  return df

def getReport(df, mes, ano, asesor):
  reporte = df.loc[
      (df['PERIODO'] == f'01/{mes}/{ano}')
      & (df['NOMBRE ASESOR(A)'].str.contains(asesor))
      #& (df['ESTADO DE INSCRIPCIÓN'].str.lower() == 'completo')
  ]
  x, y = reporte.shape
  reporte = reporte.sort_values('UNIDAD DE NEGOCIO CORRESPONDIENTE', ascending=True)

  reporte.reset_index(drop=True, inplace=True)
  reporte.index += 1
  reporte['N°'] = reporte.index
  col_indices = reporte['N°']
  reporte = reporte.drop(columns=['N°'])
  reporte.insert(0, 'N°', col_indices)
  reporte.reset_index(drop=True)

  eliminar = df.columns[df.columns.get_loc('FECHA DE INSCRIPCION PORTAL'):df.columns.get_loc('TOTAL COMISION') + 1]
  documentos = reporte.drop(columns=eliminar)
  eliminar = df.columns[df.columns.get_loc('CI'):df.columns.get_loc('CURRICULUM DOCUMENTADO') + 1]
  reporte = reporte.drop(columns=eliminar)
  if x > 0:
    asesor = reporte['NOMBRE ASESOR(A)'][1].title()
    periodo = datetime.strptime(reporte['PERIODO'][1], "%d/%m/%Y").strftime("mes de %B del %Y")
    periodo = translator.translate(periodo, src='en', dest='es').text
  else:
    asesor = asesor
    periodo = f'01/{mes}/{ano}'

  reporte = reporte.drop(columns=['NOMBRE ASESOR(A)', 'PERIODO', 'ESTADO DE INSCRIPCIÓN'])
  documentos = documentos.drop(columns=['NOMBRE ASESOR(A)', 'PERIODO'])

  return (reporte, documentos, asesor, periodo)

def getBusinesses(reporte):
  unidades_negocio = reporte['UNIDAD DE NEGOCIO CORRESPONDIENTE'].unique().tolist()
  aux = reporte['UNIDAD DE NEGOCIO CORRESPONDIENTE'].tolist()
  indexes = [aux.index(unidad) for unidad in unidades_negocio]
  indexes.append(len(aux))
  indexes = [(indexes[i], indexes[i + 1]) for i in range(0, len(indexes) - 1)]
  return indexes, unidades_negocio

def getHeaders(reporte, documentos):
  headers_r = reporte.columns.tolist()
  headers_r = headers_r[2:]
  headers_d = documentos.columns.tolist()
  headers_d = headers_d[3:]
  return (headers_r, headers_d)

def getFormatedReport(headers, reporte, documentos, indexes, unidades_negocio):
  reporte_dict = reporte.to_dict(orient='index')
  reporte_list = []
  for _, elemento in reporte_dict.items():
    nuevo = {'N°': elemento['N°'], 'Unidad de Negocio': elemento['UNIDAD DE NEGOCIO CORRESPONDIENTE']}
    lista = [x for _, x in elemento.items()]
    lista = lista[2:]
    nuevo['lista'] = lista
    reporte_list.append(nuevo)

  documentos_dict = documentos.to_dict(orient='index')
  documentos_list = []
  for _, item in documentos_dict.items():
    nuevo = {'N°': item['N°']}
    lista = [x for _, x in item.items()]
    lista = lista[3:]
    nuevo['lista'] = lista
    documentos_list.append(nuevo)

  reporte_tup = []
  for x, y in indexes:

    sublista = reporte_list[x:y]
    subdocumentos= documentos_list[x:y]
    un_negocio = unidades_negocio.pop(0)

    for fila in sublista:
      if fila['Unidad de Negocio'] != un_negocio:
        raise ValueError('Disparidad de unidades de negocio')
      del fila['Unidad de Negocio']

    reporte_tup.append((headers, un_negocio, [(item['N°'], item['lista']) for item in sublista], [(item['N°'], item['lista']) for item in subdocumentos]))

  return reporte_tup

def printInforme(datos, asesor):
  folder = 'reportes'
  template = DocxTemplate('PlantillaInformeMKT.docx')

  ih1 = InlineImage(template, image_descriptor=f'{folder}/h1({asesor}).png', width=Mm(150), height=Mm(75))
  ih2 = InlineImage(template, image_descriptor=f'{folder}/h2({asesor}).png', width=Mm(150), height=Mm(75))
  datos['historico1'] = ih1
  datos['historico2'] = ih2
  template.render(datos)
  template.save(f'{folder}/ReporteMKT({asesor}).docx')
  #files.download(f'{folder}/ReporteMKT({asesor}).docx')
  print(f'Informe generado de: {asesor}')

def enviarInforme(asesor, to_addrs, periodo):
  from_addr = 'auto.dbs@dbussinesschool.com'
  subject = f'Informe {periodo}'
  content = f'Hola {asesor} le enviamos su informe como archivo adjunto'

  msg = MIMEMultipart()
  msg['From'] = from_addr
  msg['To'] = to_addrs[0]
  msg['Subject'] = subject
  body = MIMEText(content, 'plain')
  msg.attach(body)

  archivo = f'ReporteMKT({asesor}).docx'
  path = f'reportes/{archivo}'

  pdf = MIMEApplication(open(path, 'rb').read())
  pdf.add_header('Content-Disposition', 'attachment', filename=archivo)
  msg.attach(pdf)

  server = smtplib.SMTP('smtp.gmail.com', 25)
  server.connect('smtp.gmail.com', 587)
  server.verify(from_addr)
  server.ehlo()
  server.starttls()
  server.ehlo()
  server.login(from_addr, 'Auto,dbs12\'')

  server.sendmail(from_addr, to_addrs, msg.as_string())
  server.quit()
  print(f'Informe enviado a: {to_addrs[0]}')

def getHistorical(df_ventas, asesor, ano):
  periodos = df_ventas.loc[:,['NOMBRE ASESOR(A)', 'PERIODO', 'NOMBRE PROGRAMA']]
  periodosActuales = periodos.loc[periodos['NOMBRE ASESOR(A)'].str.contains(asesor.upper()) & periodos['PERIODO'].str.contains(ano)]
  periodosPasados = periodos.loc[periodos['NOMBRE ASESOR(A)'].str.contains(asesor.upper()) & periodos['PERIODO'].str.contains(str(int(ano) - 1))]
  return (auxHistoricos(periodosActuales), auxHistoricos(periodosPasados))

def auxHistoricos(periodos):
  historico = pd.DataFrame(columns=['Mes', 'Cursos', 'Diplomados', 'Especialidades', 'Maestrías'])

  meses = range(1, 13)

  nombres_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

  for mes, nombre_mes in zip(meses, nombres_meses):
      programas_mes = periodos[periodos['PERIODO'].str.split('/').str[1].astype(int) == mes]

      cursos = programas_mes[programas_mes['NOMBRE PROGRAMA'].str.startswith('C')].shape[0]
      diplomados = programas_mes[programas_mes['NOMBRE PROGRAMA'].str.startswith('D')].shape[0]
      especialidades = programas_mes[programas_mes['NOMBRE PROGRAMA'].str.lower().str.contains('especialidad')].shape[0]
      maestrias = programas_mes[programas_mes['NOMBRE PROGRAMA'].str.startswith('M')].shape[0]

      historico.loc[mes] = [nombre_mes, cursos, diplomados, especialidades, maestrias]

  return historico

def deleteSpaces(texto):
  return re.sub(r'\s+', ' ', texto)

def printGraph(df, i, asesor, ano):
  plt.plot(df['Mes'], df['Cursos'], label='Cursos')
  plt.plot(df['Mes'], df['Diplomados'], label='Diplomados')
  plt.plot(df['Mes'], df['Especialidades'], label='Especialidades')
  plt.plot(df['Mes'], df['Maestrías'], label='Maestrías')

  for index, row in df.iterrows():
    plt.text(index - 1, row['Cursos'], str(row['Cursos']), ha='center', va='bottom')
    plt.text(index - 1, row['Diplomados'], str(row['Diplomados']), ha='center', va='bottom')
    plt.text(index - 1, row['Especialidades'], str(row['Especialidades']), ha='center', va='bottom')
    plt.text(index - 1, row['Maestrías'], str(row['Maestrías']), ha='center', va='bottom')

  plt.xticks(rotation=45)
  plt.xlabel('Meses')
  plt.ylabel('Ventas por tipo de programa')
  plt.title(f'Histórico de ventas por programa {ano}')

  plt.legend()
  plt.savefig(f'reportes/h{i}({asesor.title()}).png')
  plt.close()
  #plt.show()

def historicTables(df):
  columnas = df.iloc[0]
  df.columns = columnas

  return df[1:]

In [ ]:
#gdown.download('https://docs.google.com/document/d/1LvHDadpqLlR6qEpbq1eu7Srnye0XwS8E/edit?usp=sharing&ouid=117379727352288078520&rtpof=true&sd=true',
#               'PlantillaReporteMKT',quiet=False)
#files.download('PlantillaReporteMKT.docx')

!rm 'PlantillaInformeMKT.docx'
!mkdir reportes
file_ref = files.upload()

mkdir: cannot create directory ‘reportes’: File exists


Saving PlantillaInformeMKT.docx to PlantillaInformeMKT.docx


In [ ]:
#sede = 'DBS REPORTE DE VENTAS'
sede = 'DBS LA PAZ REPORTE DE VENTAS'
df_ventas = getDataFrame(sede, 'REGISTRO')
df_resagados = getDataFrame(sede, 'RESAGADOS')

In [ ]:
!ls reportes/

In [ ]:
_ano = '2024'
_mes = '11'
_asesor = 'NAHELLY LIBERTAD VARGAS QUISPE'.upper()

!rm -rf reportes/*

reporte, documentos, asesor, periodo = getReport(df_ventas, _mes, _ano, _asesor)
resagados, documentosr, asesorr, periodor = getReport(df_resagados, _mes, _ano, _asesor)

rep_i, rep_un = getBusinesses(reporte)
res_i, res_un = getBusinesses(resagados)

rep_headers = getHeaders(reporte, documentos)
res_headers = getHeaders(resagados, documentosr)

rep = getFormatedReport(rep_headers, reporte, documentos, rep_i, rep_un)
res = getFormatedReport(res_headers, resagados, documentosr, res_i, res_un)

h1,h2 = getHistorical(df_ventas, asesor, _ano)

datos = {
    'asesor': asesor.title(),
    'periodo': periodo,
    'h1': historicTables(h1.T),
    'h2': historicTables(h2.T),
    'fecha': translator.translate(datetime.now().strftime("%d de %B del %Y"), src='en', dest='es').text,
    'observaciones': 'Reporte generado automáticamente: (agregar su interpretación personal aquí)',
    'lista': rep,
    'resagados': res
}

printGraph(h1, 1, asesor, datetime.now().strftime("%Y"))
printGraph(h2, 2, asesor, int(datetime.now().strftime("%Y")) - 1)

printInforme(datos, datos['asesor'])

addrs = pd.DataFrame(getEMails(sede))
to_addrs = addrs.loc[addrs['Nombre'].str.lower() == asesor.lower()]['eMail'].tolist()
enviarInforme(datos['asesor'], to_addrs, periodo)

Informe generado de: Nahelly Libertad Vargas Quispe
                             Nombre                                   eMail
0  VALQUIRIA ALEXANDRA GARCIA MAYTA    valquiria.garcia@dbussinesschool.com
1     RODRIGO MARTÍN FLORES RAMÍREZ      rodrigo.flores@dbussinesschool.com
2              JENNY MAMANI HUALLPA        jenny.mamani@dbussinesschool.com
3   FERNANDO GERALD VELASQUEZ ORTIZ  fernando.velasquez@dbussinesschool.com
4       JOEL MARCELO MIYAUCHI MAYTA       joel.miyauchi@dbussinesschool.com
5    NAHELLY LIBERTAD VARGAS QUISPE      nahelly.vargas@dbussinesschool.com
['nahelly.vargas@dbussinesschool.com']
Informe enviado a: nahelly.vargas@dbussinesschool.com


In [ ]:
getEMails(sede).values.tolist()

[['VALQUIRIA ALEXANDRA GARCIA MAYTA', 'valquiria.garcia@dbussinesschool.com'],
 ['RODRIGO MARTÍN FLORES RAMÍREZ', 'rodrigo.flores@dbussinesschool.com'],
 ['JENNY MAMANI HUALLPA', 'jenny.mamani@dbussinesschool.com'],
 ['FERNANDO GERALD VELASQUEZ ORTIZ', 'fernando.velasquez@dbussinesschool.com'],
 ['JOEL MARCELO MIYAUCHI MAYTA', 'joel.miyauchi@dbussinesschool.com'],
 ['NAHELLY LIBERTAD VARGAS QUISPE', 'nahelly.vargas@dbussinesschool.com']]

In [ ]:
asesores = getEMails(sede).values.tolist()
_ano = '2024'
_mes = '11'

!rm -rf reportes/*

for _asesor, eMail in asesores:
  reporte, documentos, asesor, periodo = getReport(df_ventas, _mes, _ano, _asesor)
  resagados, documentosr, asesorr, periodor = getReport(df_resagados, _mes, _ano, _asesor)

  x, y = reporte.shape
  i, j = resagados.shape
  if x <= 0 and i <= 0:
    #print(f'Informe vacío de: {asesor.title()}')
    continue

  rep_i, rep_un = getBusinesses(reporte)
  res_i, res_un = getBusinesses(resagados)

  rep_headers = getHeaders(reporte, documentos)
  res_headers = getHeaders(resagados, documentosr)

  rep = getFormatedReport(rep_headers, reporte, documentos, rep_i, rep_un)
  res = getFormatedReport(res_headers, resagados, documentosr, res_i, res_un)

  h1,h2 = getHistorical(df_ventas, asesor, _ano)

  datos = {
    'asesor': asesor.title(),
    'periodo': periodo,
    'h1': historicTables(h1.T),
    'h2': historicTables(h2.T),
    'fecha': translator.translate(datetime.now().strftime("%d de %B del %Y"), src='en', dest='es').text,
    'observaciones': 'Reporte generado automáticamente: (agregar su interpretación personal aquí)',
    'lista': rep,
    'resagados': res
  }

  printGraph(h1, 1, asesor, datetime.now().strftime("%Y"))
  printGraph(h2, 2, asesor, int(datetime.now().strftime("%Y")) - 1)

  printInforme(datos, datos['asesor'])
  enviarInforme(datos['asesor'], [eMail], periodo)

Informe generado de: Mariela Tapia Salazar
Informe enviado a: mariela.tapia@dbussinesschool.com
Informe generado de: Margot Maldonado Yucra
Informe enviado a: margot.maldonado@dbussinesschool.com
Informe generado de: Carolina Montaño Mamani
Informe enviado a: carolina.montano@dbussinesschool.com
Informe generado de: Jean Carla Maldonado Yucra
Informe enviado a: jean.maldonado@dbussinesschool.com
Informe generado de: Nelida Ipurani Gabriel
Informe enviado a: nelida.ipurani@dbussinesschool.com
Informe generado de: Hugo Frank Aguilar Alarcón
Informe enviado a: hugo.aguilar@dbussinesschool.com
Informe generado de: Sharon Sarai Alvaro Pinola
Informe enviado a: sharon.alvaro@dbussinesschool.com
Informe generado de: Rodrigo Chalar Arnez
Informe enviado a: rodrigo.chalar@dbussinesschool.com
Informe generado de: Brandon Montenegro Zapata
Informe enviado a: brandon.montenegro@dbussinesschool.com
Informe generado de: Guadalupe Zerna Rojas
Informe enviado a: guadalupe.zerna@dbussinesschool.com
Inf

In [ ]:
#files.download('PlantillaInformeMKT.docx')
!mkdir reportes
!ls

 PlantillaInformeMKT.docx		    reportes
'ReporteMKT(mes de agosto del 2023).docx'   sample_data


In [ ]:
!ls reportes
!ls

'h1(Mariela Tapia Salazar).png'  'h2(Mariela Tapia Salazar).png'
PlantillaInformeMKT.docx  reportes  sample_data
